<a id='top'></a>
### Table of contents

1. Importing Visualization Libraries and Data

2. [Adding capital/non_capital flag to the data](#flag)

In [27]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from scipy import stats
from scipy.stats import pearsonr
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import scale
from pylab import rcParams
import matplotlib.ticker as mt
from matplotlib.ticker import ScalarFormatter
from fuzzywuzzy import process, fuzz
from datetime import date
from datetime import datetime

In [28]:
path=r'C:\Users\frauz\Documents\Python Projects\Final Project\Data\Data Prepared' #creating a path
path1=r'C:\Users\frauz\Documents\Python Projects\Final Project\Data' #creating a path

<a id='flag'></a>
### Adding capital/non_capital flag to the data

[Back to top](#top)

Using cleaned, merged, not grouped data without new variables, I flag price records made in capitals vs outside of capitals.

In [29]:
df_prices= pd.read_pickle(os.path.join(path, 'food_prices_market_unaggregated.pkl'))
df_capitals= pd.read_csv(os.path.join(path, 'countries_markets_capitals.csv'))

In [30]:
df_prices.head()

,date,market,latitude,longitude,category,currency,iso,measure_unit,product_name,price_unit,usdprice_unit
2,2000-01-15,Hirat,34.346944,62.198333,cereals and tubers,AFN,AFG,KG,Bread,15.63,0.3340
3,2000-01-15,Hirat,34.346944,62.198333,cereals and tubers,AFN,AFG,KG,Wheat,9.13,0.1951
4,2000-01-15,Hirat,34.346944,62.198333,cereals and tubers,AFN,AFG,KG,Wheat flour,10.06,0.2150
6,2000-01-15,Kabul,34.516667,69.183333,cereals and tubers,AFN,AFG,KG,Bread,14.26,0.3048
7,2000-01-15,Kabul,34.516667,69.183333,cereals and tubers,AFN,AFG,KG,Wheat,13.75,0.2939


In [31]:
df_capitals.head()

,Column1,iso,country,market,capital_market,latitude,longitude,Capital
0,0,AFG,Afghanistan,Fayzabad,non_capital,37.116638,70.580022,Kabul
1,1,AFG,Afghanistan,Hirat,non_capital,34.346944,62.198333,Kabul
2,2,AFG,Afghanistan,Jalalabad,non_capital,34.426111,70.451389,Kabul
3,3,AFG,Afghanistan,Kabul,capital,34.516667,69.183333,Kabul
4,4,AFG,Afghanistan,Kandahar,non_capital,31.612500,65.709444,Kabul


In [32]:
df_prices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2278815 entries, 2 to 3541113
Data columns (total 11 columns):
 #   Column         Dtype         
---  ------         -----         
 0   date           datetime64[ns]
 1   market         object        
 2   latitude       float64       
 3   longitude      float64       
 4   category       object        
 5   currency       object        
 6   iso            object        
 7   measure_unit   object        
 8   product_name   object        
 9   price_unit     float64       
 10  usdprice_unit  float64       
dtypes: datetime64[ns](1), float64(4), object(6)
memory usage: 208.6+ MB


In [33]:
df_capitals.duplicated().sum() #dataframe countains duplicates

0

In [34]:
df_capitals.drop_duplicates(inplace=True)
df_capitals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3672 entries, 0 to 3671
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Column1         3672 non-null   int64  
 1   iso             3672 non-null   object 
 2   country         3672 non-null   object 
 3   market          3672 non-null   object 
 4   capital_market  3672 non-null   object 
 5   latitude        3672 non-null   float64
 6   longitude       3672 non-null   float64
 7   Capital         3672 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 258.2+ KB


In [35]:
# Merging dataframes to add maret status

df_full=df_prices.merge(df_capitals[['iso','market','capital_market']], how='left', on=['iso','market'])
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2288278 entries, 0 to 2288277
Data columns (total 12 columns):
 #   Column          Dtype         
---  ------          -----         
 0   date            datetime64[ns]
 1   market          object        
 2   latitude        float64       
 3   longitude       float64       
 4   category        object        
 5   currency        object        
 6   iso             object        
 7   measure_unit    object        
 8   product_name    object        
 9   price_unit      float64       
 10  usdprice_unit   float64       
 11  capital_market  object        
dtypes: datetime64[ns](1), float64(4), object(7)
memory usage: 227.0+ MB


In [36]:
df_full.isnull().sum()

date                  0
market                0
latitude          20122
longitude         20122
category              0
currency              0
iso                   0
measure_unit          0
product_name          0
price_unit            0
usdprice_unit      2228
capital_market        0
dtype: int64

In [37]:
# Now I will group the data on country-markettype-product level

df_grouped=df_full.groupby(['iso',
                            'date',
                            'currency',
                            'capital_market',
                            'product_name']).agg({'price_unit':'mean','usdprice_unit':'mean'})

In [38]:
df_grouped.reset_index(inplace=True)
df_grouped.head()

,iso,date,currency,capital_market,product_name,price_unit,usdprice_unit
0,AFG,2000-01-15,AFN,capital,Bread,14.260000,0.304800
1,AFG,2000-01-15,AFN,capital,Wheat,13.750000,0.293900
2,AFG,2000-01-15,AFN,capital,Wheat flour,18.570000,0.396900
3,AFG,2000-01-15,AFN,non_capital,Bread,15.580000,0.332967
4,AFG,2000-01-15,AFN,non_capital,Wheat,11.723333,0.250567


In [39]:


df_grouped.duplicated().value_counts()

False    193836
dtype: int64

In [40]:
df_grouped.duplicated().value_counts()

False    193836
dtype: int64

In [41]:
df_grouped.loc[df_grouped['price_unit'].idxmin()]

iso                               ZWE
date              2018-12-15 00:00:00
currency                          USD
capital_market            non_capital
product_name                     Salt
price_unit                   0.000117
usdprice_unit                0.000117
Name: 193453, dtype: object

In [42]:
df_grouped.loc[df_grouped['usdprice_unit'].idxmin()]

iso                               BLR
date              2009-01-15 00:00:00
currency                          BYR
capital_market                capital
product_name                    Bread
price_unit                       0.25
usdprice_unit                     0.0
Name: 19405, dtype: object

In [43]:
df_grouped.loc[df_grouped['price_unit'].idxmax()]

iso                               SOM
date              2015-12-15 00:00:00
currency                          SOS
capital_market            non_capital
product_name                     Meat
price_unit                 17250000.0
usdprice_unit              29896.0139
Name: 152305, dtype: object

In [44]:
df_grouped. isnull().sum()

iso               0
date              0
currency          0
capital_market    0
product_name      0
price_unit        0
usdprice_unit     0
dtype: int64

In [45]:
# exporting the data 

df_grouped.to_pickle(os.path.join(path, 'food_prices_market_cap_noncap.pkl'))